<a href="https://colab.research.google.com/github/danielsaggau/IR_LDC/blob/main/model/SCOTUS/mean_pool_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets
from datasets import load_dataset

In [ ]:
dataset=load_dataset('lex_glue', 'scotus')

In [ ]:
from transformers import AutoTokenizer
#!python -c "from huggingface_hub.hf_api import HfFolder; HfFolder.save_token('XXXX')"
tokenizer = AutoTokenizer.from_pretrained('danielsaggau/longformer_simcse_scotus', use_auth_token=True,use_fast=True)

In [27]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained('danielsaggau/longformer_simcse_scotus',use_auth_token=True, num_labels=14)

All model checkpoint weights were used when initializing LongformerForSequenceClassification.

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at danielsaggau/longformer_simcse_scotus and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
import torch 
from torch import nn

In [29]:
# Copied from transformers.models.bert.modeling_bert.BertPooler
class CustomLongformerPooler(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()

    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
        # We "pool" the model by simply taking the hidden state corresponding
        # to the first token.
        mean_token_tensor = hidden_states.mean(dim=1)
        pooled_output = self.dense(mean_token_tensor)
        pooled_output = self.activation(pooled_output)
        return pooled_output

In [30]:
# Use custom pooler with mean-pooling instead of cls-pooling
model.longformer.pooler = CustomLongformerPooler(model.config)

In [19]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [22]:
tokenized_data = dataset.map(preprocess_function, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [10]:
from datasets import load_metric
import numpy as np

def compute_metrics(eval_pred):
    metric1 = load_metric("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    micro1 = metric1.compute(predictions=predictions, references=labels, average="micro")["f1"]
    macro1 = metric1.compute(predictions=predictions, references=labels, average="macro")["f1"]
    return { "f1-micro": micro1, "f1-macro": macro1}

In [11]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8) # fp16

In [ ]:
!git clone https://github.com/danielsaggau/IR_LDC.git

In [13]:
%cd IR_LDC

/content/IR_LDC


In [ ]:
!pip install wandb
import wandb
wandb.login()
#fd6f7deb3126d40be9abf77ee753bf45f00e2a9a

In [31]:
wandb.init(project="IR_LDC")
wandb.init(name="mean_linear")

eval/f1-macro,▁▄▅▅▅▅▆▆▆▆▇▇▇▇██████▇
eval/f1-micro,▁▄▅▅▆▆▇▇▇▇▇██████████
eval/loss,█▅▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▁▂▃▃▅▅▄▆▄▅▃▄▅▃▄▄▃▄▄▄█
eval/samples_per_second,█▇▆▅▄▄▅▃▅▄▅▅▄▆▅▅▆▅▅▅▁
eval/steps_per_second,█▇▆▅▄▄▅▃▄▄▅▄▄▆▅▅▆▅▅▅▁
train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/learning_rate,███▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▁▁▁
train/loss,█▆▄▃▃▃▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁


In [35]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="/scotus_mean",
    learning_rate=3e-5,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    push_to_hub=True,
    report_to="wandb",
    fp16=True,
    run_name="mean",
    metric_for_best_model="f1-micro",
    greater_is_better=True,
    load_best_model_at_end = True
)

PyTorch: setting up devices


In [36]:
from transformers import Trainer, EarlyStoppingCallback
import torch
trainer = Trainer(
    model=model,
    compute_metrics=compute_metrics,
    args=training_args,
    eval_dataset=tokenized_data['test'],
    train_dataset=tokenized_data["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,    
    callbacks = [EarlyStoppingCallback(early_stopping_patience=10)])
trainer.train()

/scotus_mean is already a clone of https://huggingface.co/danielsaggau/scotus_mean. Make sure you pull the latest changes with `repo.git_pull()`.
Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `LongformerForSequenceClassification.forward` and have been ignored: text. If text are not expected by `LongformerForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 5000
  Num Epochs = 10
  Instantaneous batch size per device = 6
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization ste

Epoch,Training Loss,Validation Loss,F1-micro,F1-macro
1,0.925200,0.950030,0.715000,0.547118
2,0.607800,1.049444,0.732857,0.621909
3,0.463700,1.040940,0.754286,0.672185
4,0.320900,1.224971,0.760714,0.684528
5,0.195600,1.495215,0.753571,0.664774
6,0.144800,1.558114,0.759286,0.671938
7,0.077500,1.592752,0.763571,0.682401
8,0.063200,1.627468,0.768571,0.684947
9,0.039000,1.714876,0.762143,0.683931
10,0.021500,1.705329,0.762857,0.683185


Streaming output truncated to the last 5000 lines.
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention

TrainOutput(global_step=8340, training_loss=0.2801493309670501, metrics={'train_runtime': 7985.8209, 'train_samples_per_second': 6.261, 'train_steps_per_second': 1.044, 'total_flos': 2.969597381069664e+16, 'train_loss': 0.2801493309670501, 'epoch': 10.0})

In [38]:
trainer.evaluate(eval_dataset=tokenized_data['validation'])

The following columns in the evaluation set don't have a corresponding argument in `LongformerForSequenceClassification.forward` and have been ignored: text. If text are not expected by `LongformerForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1400
  Batch size = 6
Initializing global attention on CLS token...


Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on C

{'eval_loss': 1.6274678707122803,
 'eval_f1-micro': 0.7685714285714286,
 'eval_f1-macro': 0.684947481074309,
 'eval_runtime': 64.2551,
 'eval_samples_per_second': 21.788,
 'eval_steps_per_second': 3.642,
 'epoch': 10.0}